In [ ]:
#!pip install langchain

In [6]:
#!pip install -qU langchain-openai

In [ ]:
#!pip install langchain_community

In [1]:
import os
from openai import AzureOpenAI

### Credentials

In [3]:
apikey = 'key'
endpoint = 'endpoint'
deployname = 'gpt-4-2-teste'
apiversion =  '2024-08-01-preview'

In [4]:
client = AzureOpenAI(
    api_key=apikey,
    azure_endpoint=endpoint,
    api_version=apiversion
)

In [5]:
chat_completion = client.chat.completions.create(
    messages=[
        {"role":"user",
        "content":"Oi"}
    ],
    model=deployname,
)

print(chat_completion.choices[0].message.content)

Olá! Como posso ajudar você hoje?


### Langchain

In [8]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [13]:
#Configurar o modelo no LangChain
model = AzureChatOpenAI(
    api_key=apikey,
    model=deployname,
    azure_endpoint=endpoint,
    api_version=apiversion
)

In [14]:
# Criar mensagens usando LangChain
messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

In [15]:
# Chamar o modelo usando invoke
response = model.invoke(messages)

In [16]:
print(response.content)

Ciao!


In [17]:
#LangChain também tem suporte a input via string
model.invoke("Hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_5603ee5e2e', 'finish_reason': 'stop', 'logprobs': None}, id='run-6d887a49-a138-4757-adee-a4c5dc734e5a-0')

In [19]:
model.invoke([HumanMessage("Hello")])

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_5603ee5e2e', 'finish_reason': 'stop', 'logprobs': None}, id='run-2d8604d5-0fa6-42d7-8777-8e932d81aa6e-0')

##### Streaming

Refere-se ao processo de enviar as respostas do modelo em partes, ou tokens individuais, em vez de aguardar que o modelo gere toda a resposta antes de enviá-la. Isso é útil em situações onde você deseja exibir resultados parciais para o usuário em tempo real ou processar a saída à medida que ela é gerada.

In [20]:
for token in model.stream(messages):
    print(token.content, end="|")

|C|iao|!||

##### Prompt Templates

O conceito de Prompt Templates no LangChain refere-se a uma ferramenta que ajuda a estruturar e formatar as mensagens de entrada para que estejam prontas para serem enviadas ao modelo de linguagem. Isso é particularmente útil quando as mensagens precisam ser personalizadas com base em variáveis fornecidas pelo usuário (como entradas dinâmicas) e transformadas em um formato específico.

In [21]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [22]:
#A entrada para este modelo de prompt é um dicionário
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [23]:
#Se quisermos acessar as mensagens diretamente, fazemos
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

In [24]:
#Podemos invocar o modelo de chat no prompt formatado
response = model.invoke(prompt)
print(response.content)

Ciao!
